In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import mysql.connector
import time

In [2]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="job_scraping_database"
)

cursor = mydb.cursor()


In [3]:
# Define chrome_options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=./chromedriver.exe")

In [4]:
wd = webdriver.Chrome(options = chrome_options)
time.sleep(1)
wd.get("https://www.glassdoor.com/Job/united-states-marketing-jobs-SRCH_IL.0,13_IN1_KO14,23.htm?seniorityType=entrylevel")
time.sleep(3)
# Simulate a click anywhere on the screen
action_chains = ActionChains(wd)
action_chains.move_by_offset(100, 100).click().perform()

In [ ]:
page_number = 1
def glassdoor_scraper():
    global page_number
    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("li", class_ = "react-job-listing css-108gl9c eigr9kq3")
    
    for job in jobs:
        #JobId
        job_id = job.get("data-id")
        
        #Check if the JobId already exists in the table
        select_query = "SELECT JobId FROM Marketing_Jobs WHERE JobId = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()
        
        if result is None:
            #Job Title
            job_title = job.find("div", class_ = "job-title mt-xsm").text.strip()
            
            #Job Link
            job_link = "https://www.glassdoor.com" + job.find("a", class_ = "d-flex justify-content-between p-std jobCard")["href"]
            
            #Company Name
            company_name_elem = job.find("div", class_ = "job-search-8wag7x")
            company_name = company_name_elem.get_text(strip = True, separator = " ") if company_name_elem else ""
            if "★" in company_name:
                company_name = company_name.rsplit(" ", 2)[0]
            
            #Location
            location = job.find("div", class_ = "location mt-xxsm").text.strip()
            
            driver = webdriver.Chrome(options = chrome_options)
            time.sleep(1)
            driver.get(job_link)
            time.sleep(3)
            job_src = driver.page_source
            job_soup = bs(job_src, "html.parser")
            
            #Salary
            salary_elem = job_soup.find("span", class_ = "small css-10zcshf e1v3ed7e1") 
            salary = salary_elem.text.strip() if salary_elem else ""
            if ":" in salary:
                salary = salary.split(":")[1]
            elif "(" in salary:
                salary = salary.split("(")[0]
            
            #Job Description
            job_des_elem = job_soup.find("div", class_ = "desc css-58vpdc ecgq1xb5")
            job_des = job_des_elem.text.strip() if job_des_elem else ""
            
            #Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO Marketing_Jobs (JobId, Title, Company, Location, Apply, Salary, JobDescription)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
            driver.quit()
        else:
            print("Skipping duplicate JobId:", job_id)
    print("Data updated for page:", page_number)
    page_number += 1
    try:
        next_button = wd.find_element(By.XPATH, "//button[@class = 'nextButton job-search-opoz2d e13qs2072']")
        next_button.click()
        time.sleep(3)
        try:
            x_button = wd.find_element(By.XPATH, "//button[@class = 'e1jbctw80 ei0fd8p1 css-1n14mz9 e1q8sty40']")
            x_button.click()
        except NoSuchElementException:
            print("")
        glassdoor_scraper()
    except NoSuchElementException:
        cursor.close()
        mydb.close()
        wd.quit()
glassdoor_scraper()

Data updated for page: 1


Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


Data updated for page: 1

Skipping duplicate JobId: 1008621094138
Skipping duplicate JobId: 1008594623996
Skipping duplicate JobId: 1008787832037
Skipping duplicate JobId: 1008751262308
Data updated for page: 1

Skipping duplicate JobId: 1008748252842
Skipping duplicate JobId: 1008666491231
Skipping duplicate JobId: 1008759832506
Skipping duplicate JobId: 1008775017907
Data updated for page: 1

Skipping duplicate JobId: 1008791140256
Skipping duplicate JobId: 1008666491231
Skipping duplicate JobId: 1007362487417
Skipping duplicate JobId: 1008791506594
Skipping duplicate JobId: 1007783561076
Skipping duplicate JobId: 1008659813159
Skipping duplicate JobId: 1008785411471
Data updated for page: 1

Skipping duplicate JobId: 1008539628949
Skipping duplicate JobId: 1008593485409
Skipping duplicate JobId: 1008791200031
Skipping duplicate JobId: 1008781614288
Skipping duplicate JobId: 1006793348667
Skipping duplicate JobId: 1008790587805
Skipping duplicate JobId: 1008790667929
Skipping duplica